In [ ]:
import numpy as np
import pandas as pd


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"theraghavjuneja","key":"d24836b84ec39cad87a34aef68ea0283"}'}

In [ ]:
import os
import zipfile

# Make a hidden directory for Kaggle
os.makedirs('/root/.kaggle', exist_ok=True)

# Move the uploaded kaggle.json into the directory
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other


In [ ]:
with zipfile.ZipFile("/content/chest-xray-pneumonia.zip", 'r') as zip_ref:
    zip_ref.extractall("chestX-ray")


In [ ]:
import os

path = "/content/chestX-ray/chest_xray/train/NORMAL"

# List all items in the directory
all_items = os.listdir(path)

# Separate files and directories
num_files = sum(os.path.isfile(os.path.join(path, item)) for item in all_items)
num_dirs = sum(os.path.isdir(os.path.join(path, item)) for item in all_items)

print(f"Total items: {len(all_items)}")
print(f"Number of files: {num_files}")
print(f"Number of folders: {num_dirs}")


Total items: 1341
Number of files: 1341
Number of folders: 0


In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

img_size = 299

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1
)


val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


train_data = train_datagen.flow_from_directory(
    '/content/chestX-ray/chest_xray/train',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='binary'
)


val_data = val_datagen.flow_from_directory(
    '/content/chestX-ray/chest_xray/val',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='binary'
)


base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(train_data, validation_data=val_data, epochs=5)

# Optional: Unfreeze some layers and fine-tune further
# base_model.trainable = True
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(train_data, validation_data=val_data, epochs=5)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.8813 - loss: 0.2578 - val_accuracy: 0.8750 - val_loss: 0.2269
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 154s 943ms/step - accuracy: 0.9437 - loss: 0.1476 - val_accuracy: 0.8750 - val_loss: 0.2204
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 202s 945ms/step - accuracy: 0.9420 - loss: 0.1452 - val_accuracy: 0.9375 - val_loss: 0.1748
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 154s 945ms/step - accuracy: 0.9502 - loss: 0.1384 - val_accuracy: 0.9375 - val_loss: 0.1909
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 202s 943ms/step - accuracy: 0.9344 - loss: 0.1445 - val_accuracy: 0.9375 - val_loss: 0.1505


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data = test_datagen.flow_from_directory(
    '/content/chestX-ray/chest_xray/test',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 624 images belonging to 2 classes.


In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 320ms/step - accuracy: 0.8290 - loss: 0.3651
Test Loss: 0.2592
Test Accuracy: 0.8766


In [ ]:
model.save('xray_model.h5')
